### 영화티켓가격 크롤링

In [57]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from collections import Counter
import statistics

import datetime

In [58]:
# 조회일자 설정 2018-01 ~ 2023-04, 조회기간은 매월 1일~7일 (주중+주말 포함)

start = []
end = []
for year in range(2018, 2023):
    for month in range(1,13):
        start.append(datetime.date(year, month, 1).strftime('%Y-%m-%d'))
        end.append(datetime.date(year, month, 7).strftime('%Y-%m-%d'))
for month in range(1,5):
    start.append(datetime.date(2023, month, 1).strftime('%Y-%m-%d'))
    end.append(datetime.date(2023, month, 7).strftime('%Y-%m-%d'))
len(start)

64

In [4]:
# 최빈값 함수
def modefinder(numbers):
    c = Counter(numbers)
    mode = c.most_common(1)
    return mode[0][0]

In [5]:
base_url = 'https://www.kobis.or.kr/kobis/business/mast/thea/findShowHistorySc.do'
driver = webdriver.Chrome()
driver.get(base_url)
time.sleep(3)

# 지역선택
driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/div[2]/div[1]/label').click()
driver.find_element(By.XPATH, '//*[@id="sWideareaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="sWideareaCd"]/option[2]').click()  #서울시
time.sleep(0.2)

In [43]:
### CGV : 홍대점
min1 = []
max1 = []
mode1 = []
median1 = []

driver.find_element(By.XPATH, '//*[@id="sBasareaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="sBasareaCd"]/option[14]').click()  #마포구
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="sTheaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="sTheaCd"]/option[4]').click()  #CGV홍대
time.sleep(0.2)
for i in tqdm(range(0,len(start))):
    # 상영기간 선택
    driver.find_element(By.XPATH, '//*[@id="showStartDt"]').clear()
    driver.find_element(By.XPATH, '//*[@id="showStartDt"]').send_keys(start[i])
    driver.find_element(By.XPATH, '//*[@id="showEndDt"]').clear()
    driver.find_element(By.XPATH, '//*[@id="showEndDt"]').send_keys(end[i])
    time.sleep(0.2)
    driver.find_element(By.XPATH, '//*[@id="btn_search"]').click()
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list = soup.select('table.tbl3.info3 > tbody > tr > td.left')
    price = []
    for i in range(0, len(list)+1):
        try:
            lis = soup.select('table.tbl3.info3 > tbody > tr > td.left')[i]
            pr = int(lis.text.split(" ")[1].split('(')[1].split('원')[0].replace(',',''))
            if pr != 0:
                price.append(pr)
        except:
            pass
    min1.append(min(price))
    max1.append(max(price))
    mode1.append(modefinder(price))
    median1.append(statistics.median(price))
min1 = pd.DataFrame(min1, columns = ['cgv_min'])
max1 = pd.DataFrame(max1, columns = ['cgv_max'])
mode1 = pd.DataFrame(mode1, columns = ['cgv_mode'])
median1 = pd.DataFrame(median1, columns = ['cgv_median'])
cgv = pd.concat([min1, median1, max1, mode1], axis = 1)
cgv = cgv.astype({'cgv_median':'int64'})
cgv.to_csv('CGV.csv', index = False)

  0%|          | 0/64 [00:00<?, ?it/s]

In [47]:
### 롯데시네마: 명동 에비뉴엘점
min2 = []
max2 = []
mode2 = []
median2 = []

driver.find_element(By.XPATH, '//*[@id="sBasareaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="sBasareaCd"]/option[25]').click()  #중구
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="sTheaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="sTheaCd"]/option[9]').click()  #롯데시네마 에비뉴엘(명동)
time.sleep(0.2)

for i in tqdm(range(0,len(start))):
    # 상영기간 선택
    driver.find_element(By.XPATH, '//*[@id="showStartDt"]').clear()
    driver.find_element(By.XPATH, '//*[@id="showStartDt"]').send_keys(start[i])
    driver.find_element(By.XPATH, '//*[@id="showEndDt"]').clear()
    driver.find_element(By.XPATH, '//*[@id="showEndDt"]').send_keys(end[i])
    time.sleep(0.2)
    driver.find_element(By.XPATH, '//*[@id="btn_search"]').click()
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list = soup.select('table.tbl3.info3 > tbody > tr > td.left')
    price = []
    for i in range(0, len(list)+1):
        try:
            lis = soup.select('table.tbl3.info3 > tbody > tr > td.left')[i]
            pr = int(lis.text.split(" ")[1].split('(')[1].split('원')[0].replace(',',''))
            if pr != 0:
                price.append(pr)
        except:
            pass
    min2.append(min(price))
    max2.append(max(price))
    mode2.append(modefinder(price))
    median2.append(statistics.median(price))
min2 = pd.DataFrame(min2, columns = ['lotte_min'])
max2 = pd.DataFrame(max2, columns = ['lotte_max'])
mode2 = pd.DataFrame(mode2, columns = ['lotte_mode'])
median2 = pd.DataFrame(median2, columns = ['lotte_median'])
lotte = pd.concat([min2, median2, max2, mode2], axis = 1)
lotte = lotte.astype({'lotte_median':'int64'})
lotte.to_csv('LotteCinema.csv', index = False)

  0%|          | 0/64 [00:00<?, ?it/s]

In [8]:
### 메가박스: 코엑스점
min3 = []
max3 = []
mode3 = []
median3 = []

driver.find_element(By.XPATH, '//*[@id="sBasareaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="sBasareaCd"]/option[2]').click()  #강남구
time.sleep(0.2)
driver.find_element(By.XPATH, '//*[@id="sTheaCd"]').click()
driver.find_element(By.XPATH, '//*[@id="sTheaCd"]/option[11]').click()  #메가박스 코엑스
time.sleep(0.2)

for i in tqdm(range(0,len(start))):
    # 상영기간 선택
    driver.find_element(By.XPATH, '//*[@id="showStartDt"]').clear()
    driver.find_element(By.XPATH, '//*[@id="showStartDt"]').send_keys(start[i])
    driver.find_element(By.XPATH, '//*[@id="showEndDt"]').clear()
    driver.find_element(By.XPATH, '//*[@id="showEndDt"]').send_keys(end[i])
    time.sleep(0.2)
    driver.find_element(By.XPATH, '//*[@id="btn_search"]').click()
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list = soup.select('table.tbl3.info3 > tbody > tr > td.left')
    price = []
    for i in range(0, len(list)+1):
        try:
            lis = soup.select('table.tbl3.info3 > tbody > tr > td.left')[i]
            pr = int(lis.text.split(" ")[1].split('(')[1].split('원')[0].replace(',',''))
            if pr != 0:
                price.append(pr)
        except:
            pass
    min3.append(min(price))
    max3.append(max(price))
    mode3.append(modefinder(price))
    median3.append(statistics.median(price))
min3 = pd.DataFrame(min3, columns = ['mega_min'])
max3 = pd.DataFrame(max3, columns = ['mega_max'])
mode3 = pd.DataFrame(mode3, columns = ['mega_mode'])
median3 = pd.DataFrame(median3, columns = ['mega_median'])
megabox = pd.concat([min3, median3, max3, mode3], axis = 1)
megabox = megabox.astype({'mega_median':'int64'})
megabox.to_csv('Megabox.csv', index = False)

  0%|          | 0/64 [00:00<?, ?it/s]

In [92]:
cgv = pd.read_csv('CGV.csv')
lotte = pd.read_csv('LotteCinema.csv')
megabox = pd.read_csv('Megabox.csv')
cgv.index = start
lotte.index = start
megabox.index = start

In [94]:
ticket = pd.concat([cgv, lotte, megabox], axis = 1)
ticket.to_csv('TicketPrice_1.csv', index = False)
ticket.to_csv('TicketPrice_2.csv')